In [ ]:
import os
import numpy as np
import cv2  # استيراد مكتبة OpenCV
import supervision as sv
from sports.annotators.soccer import (
    draw_pitch,
    draw_points_on_pitch,
    draw_pitch_voronoi_diagram
)

# إعداد المسارات
SOURCE_VIDEO_PATH = "/content/جول.mp4"  # مسار الفيديو الجديد
BASE_OUTPUT_DIR = "F:\\khz"  # المسار الأساسي لحفظ الإطارات

# التأكد من وجود مجلدات لحفظ الإطارات في مكان جديد
os.makedirs(os.path.join(BASE_OUTPUT_DIR, 'annotated_frames_new'), exist_ok=True)
os.makedirs(os.path.join(BASE_OUTPUT_DIR, 'radar_frames_new'), exist_ok=True)
os.makedirs(os.path.join(BASE_OUTPUT_DIR, 'voronoi_frames_new'), exist_ok=True)

# إعداد المتغيرات
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3

ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=20, height=17
)

tracker = sv.ByteTrack()
tracker.reset()

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

frame_count = 0  # عداد الإطارات

for frame in frame_generator:
    # ball, goalkeeper, player, referee detection
    result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    detections = sv.Detections.from_inference(result)

    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

    all_detections = detections[detections.class_id != BALL_ID]
    all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
    all_detections = tracker.update_with_detections(detections=all_detections)

    goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
    players_detections = all_detections[all_detections.class_id == PLAYER_ID]
    referees_detections = all_detections[all_detections.class_id == REFEREE_ID]

    # team assignment
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    players_detections.class_id = team_classifier.predict(players_crops)

    goalkeepers_detections.class_id = resolve_goalkeepers_team_id(
        players_detections, goalkeepers_detections)

    referees_detections.class_id -= 1

    all_detections = sv.Detections.merge([
        players_detections, goalkeepers_detections, referees_detections
    ])

    # frame visualization
    labels = [f"#{tracker_id}" for tracker_id in all_detections.tracker_id]
    all_detections.class_id = all_detections.class_id.astype(int)

    annotated_frame = frame.copy()
    annotated_frame = ellipse_annotator.annotate(scene=annotated_frame, detections=all_detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=all_detections, labels=labels)
    annotated_frame = triangle_annotator.annotate(scene=annotated_frame, detections=ball_detections)

    # حفظ الإطار المرسوم باستخدام OpenCV في مجلد "annotated_frames_new"
    annotated_output_path = os.path.join(BASE_OUTPUT_DIR, 'annotated_frames_new', f"frame_{frame_count:04d}.png")
    cv2.imwrite(annotated_output_path, annotated_frame)  # حفظ الإطار كصورة

    frame_count += 1  # زيادة العداد لكل فريم

    # detect pitch key points
    result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(result)

    # project ball, players and referees on pitch
    filter = key_points.confidence[0] > 0.5
    frame_reference_points = key_points.xy[0][filter]
    pitch_reference_points = np.array(CONFIG.vertices)[filter]

    transformer = ViewTransformer(
        source=frame_reference_points,
        target=pitch_reference_points
    )

    frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_ball_xy = transformer.transform_points(points=frame_ball_xy)

    players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_players_xy = transformer.transform_points(points=players_xy)

    referees_xy = referees_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_referees_xy = transformer.transform_points(points=referees_xy)

    # visualize video game-style radar view
    radar_frame = draw_pitch(CONFIG)
    radar_frame = draw_points_on_pitch(
        config=CONFIG,
        xy=pitch_ball_xy,
        face_color=sv.Color.WHITE,
        edge_color=sv.Color.BLACK,
        radius=10,
        pitch=radar_frame)
    radar_frame = draw_points_on_pitch(
        config=CONFIG,
        xy=pitch_players_xy[players_detections.class_id == 0],
        face_color=sv.Color.from_hex('00BFFF'),
        edge_color=sv.Color.BLACK,
        radius=16,
        pitch=radar_frame)
    radar_frame = draw_points_on_pitch(
        config=CONFIG,
        xy=pitch_players_xy[players_detections.class_id == 1],
        face_color=sv.Color.from_hex('FF1493'),
        edge_color=sv.Color.BLACK,
        radius=16,
        pitch=radar_frame)
    radar_frame = draw_points_on_pitch(
        config=CONFIG,
        xy=pitch_referees_xy,
        face_color=sv.Color.from_hex('FFD700'),
        edge_color=sv.Color.BLACK,
        radius=16,
        pitch=radar_frame)

    # حفظ الإطار المرسوم باستخدام OpenCV في مجلد "radar_frames_new"
    radar_output_path = os.path.join(BASE_OUTPUT_DIR, 'radar_frames_new', f"radar_frame_{frame_count:04d}.png")
    cv2.imwrite(radar_output_path, radar_frame)  # حفظ الإطار كصورة

    # visualize voronoi diagram
    voronoi_frame = draw_pitch(CONFIG)
    voronoi_frame = draw_pitch_voronoi_diagram(
        config=CONFIG,
        team_1_xy=pitch_players_xy[players_detections.class_id == 0],
        team_2_xy=pitch_players_xy[players_detections.class_id == 1],
        team_1_color=sv.Color.from_hex('00BFFF'),
        team_2_color=sv.Color.from_hex('FF1493'),
        pitch=voronoi_frame)

    # حفظ الإطار المرسوم باستخدام OpenCV في مجلد "voronoi_frames_new"
    voronoi_output_path = os.path.join(BASE_OUTPUT_DIR, 'voronoi_frames_new', f"voronoi_frame_{frame_count:04d}.png")
    cv2.imwrite(voronoi_output_path, voronoi_frame)  # حفظ الإطار كصورة

print(f"تم حفظ {frame_count} إطارًا في المجلدات الجديدة.")


In [ ]:
import cv2
import os

# إعداد المسارات
BASE_INPUT_DIR = "/content/F:\\khz"  # المسار الذي يحتوي على مجلدات الفريمات الجديدة
OUTPUT_VIDEO_DIR = "/content/output_videos"  # المسار الذي سيتم حفظ الفيديوهات فيه

# التأكد من وجود مجلد لحفظ الفيديوهات
os.makedirs(OUTPUT_VIDEO_DIR, exist_ok=True)

# إعداد بيانات الفيديو الأصلي
ORIGINAL_VIDEO_DURATION = 30  # مدة الفيديو الأصلي بالثواني
ORIGINAL_VIDEO_FPS = 30  # عدد الإطارات في الثانية (FPS) للفيديو الأصلي

folders = ['annotated_frames_new', 'radar_frames_new', 'voronoi_frames_new']  # مجلدات الفريمات الجديدة

for folder in folders:
    # مسار مجلد الفريمات
    folder_path = os.path.join(BASE_INPUT_DIR, folder)
    
    # الحصول على قائمة الفريمات (sorted لضمان ترتيب الإطارات)
    images = sorted([img for img in os.listdir(folder_path) if img.endswith(".png")])

    if len(images) == 0:
        print(f"لا توجد فريمات في المجلد {folder}")
        continue

    # حساب FPS المطلوب بناءً على مدة الفيديو الأصلي
    fps = len(images) / ORIGINAL_VIDEO_DURATION

    # قراءة أول فريم للحصول على أبعاد الفيديو
    first_frame = cv2.imread(os.path.join(folder_path, images[0]))
    height, width, _ = first_frame.shape

    # إعداد الفيديو
    video_output_path = os.path.join(OUTPUT_VIDEO_DIR, f"{folder}.mp4")
    video_writer = cv2.VideoWriter(video_output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    # كتابة الفريمات للفيديو
    for image in images:
        frame_path = os.path.join(folder_path, image)
        frame = cv2.imread(frame_path)
        video_writer.write(frame)

    # إغلاق ملف الفيديو
    video_writer.release()

    print(f"تم تجميع الفيديو من {folder} وحفظه في {video_output_path}")
